# (목적)Gradient를 확인한다
## 전체 Gradient에 3을 곱한것과 parameter 별로 gradient에 3을 곱한 것이 차이가 나는지 확인

In [1]:
import torch
import torch.nn as nn

import torch.nn.functional as F

# 예제 데이터 생성
X = torch.randn(100, 2)
y = torch.randint(0, 2, (100,))

# 모델 정의
class ClassificationModel(nn.Module):
    def __init__(self):
        super(ClassificationModel, self).__init__()
        self.fc1 = nn.Linear(2, 2)
        self.fc2 = nn.Linear(2, 2)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x

model = ClassificationModel()

# Cross Entropy Loss와 KL Divergence Loss 정의
cross_entropy_loss = nn.CrossEntropyLoss()
kl_divergence_loss = nn.KLDivLoss(reduction='batchmean')

# Forward Pass
outputs = model(X)

# Cross Entropy Loss 계산
ce_loss = F.cross_entropy(outputs, y)

# KL Divergence Loss 계산
p = torch.tensor([0.1, 0.2])  # 예시 분포
kl_loss = kl_divergence_loss(torch.log(outputs), p)
# kl_loss = ce_loss

# Gradient 계산 (torch.autograd.grad 사용)
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
kl_gradient = torch.autograd.grad(kl_loss, model.parameters(), retain_graph=True)

# Gradient 확인
print("Cross Entropy Loss Gradient:")
for grad_param in ce_gradient:
    print(grad_param)

# print("\nKL Divergence Loss Gradient:")
# for grad_param in kl_gradient:
#     print(grad_param)

Cross Entropy Loss Gradient:
tensor([[ 0.0557, -0.0430],
        [-0.1193,  0.0920]])
tensor([-0.0010,  0.0022])
tensor([[-0.0689,  0.1304],
        [ 0.0689, -0.1304]])
tensor([ 0.0022, -0.0022])


In [2]:
ce_loss *= 2
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)

# Gradient 확인
print("Cross Entropy Loss Gradient:")
for grad_param in ce_gradient:
    print(grad_param)

Cross Entropy Loss Gradient:
tensor([[ 0.1115, -0.0859],
        [-0.2387,  0.1840]])
tensor([-0.0020,  0.0043])
tensor([[-0.1377,  0.2607],
        [ 0.1377, -0.2607]])
tensor([ 0.0044, -0.0044])


In [3]:
kl_loss

tensor(nan, grad_fn=<DivBackward0>)

In [4]:
total_loss = 1 * ce_loss +  2* kl_loss
total_loss
total_gradient = torch.autograd.grad(total_loss, model.parameters(), retain_graph=True)
print(total_gradient)

(tensor([[-0.2446,  0.1243],
        [ 0.5039, -0.1083]]), tensor([-0.8359,  1.1386]), tensor([[ 0.2636, -0.2024],
        [-1.3605,  0.4303]]), tensor([ 0.2386, -1.7379]))


In [5]:
names_grads_copy = {}

for param_name, ce_grad, kl_grad in zip(model.parameters(), ce_gradient, kl_gradient):
    
    #kl_grad = None
    
    if not kl_grad == None:
        names_grads_copy[param_name] = torch.tensor(1) * ce_grad + torch.tensor(2) *  kl_grad
    else:
        names_grads_copy[param_name] = torch.tensor(1) * ce_grad 
    
for key, value in names_grads_copy.items():
    print(value)

tensor([[-0.2446,  0.1243],
        [ 0.5039, -0.1083]])
tensor([-0.8359,  1.1386])
tensor([[ 0.2636, -0.2024],
        [-1.3605,  0.4303]])
tensor([ 0.2386, -1.7379])


## 1. Gradient를 추가적으로 구해도 retain_graph=True를 설정하면, 값이 변하지 않는다

In [6]:
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(grad_param)

tensor([[ 0.1115, -0.0859],
        [-0.2387,  0.1840]])
tensor([-0.0020,  0.0043])
tensor([[-0.1377,  0.2607],
        [ 0.1377, -0.2607]])
tensor([ 0.0044, -0.0044])


# 2. 전체 gradient에 3을 곱한다
## 1) 안된다

In [7]:
ce_gradient =  torch.tensor(3) * torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(grad_param)

tensor([[ 0.1115, -0.0859],
        [-0.2387,  0.1840]])
tensor([-0.0020,  0.0043])
tensor([[-0.1377,  0.2607],
        [ 0.1377, -0.2607]])
tensor([ 0.0044, -0.0044])
tensor([[ 0.1115, -0.0859],
        [-0.2387,  0.1840]])
tensor([-0.0020,  0.0043])
tensor([[-0.1377,  0.2607],
        [ 0.1377, -0.2607]])
tensor([ 0.0044, -0.0044])
tensor([[ 0.1115, -0.0859],
        [-0.2387,  0.1840]])
tensor([-0.0020,  0.0043])
tensor([[-0.1377,  0.2607],
        [ 0.1377, -0.2607]])
tensor([ 0.0044, -0.0044])


## 2) 덧셈 조차 하면 안된다

In [8]:
total_gradient = ce_gradient + kl_gradient
total_gradient

(tensor([[ 0.1115, -0.0859],
         [-0.2387,  0.1840]]),
 tensor([-0.0020,  0.0043]),
 tensor([[-0.1377,  0.2607],
         [ 0.1377, -0.2607]]),
 tensor([ 0.0044, -0.0044]),
 tensor([[ 0.1115, -0.0859],
         [-0.2387,  0.1840]]),
 tensor([-0.0020,  0.0043]),
 tensor([[-0.1377,  0.2607],
         [ 0.1377, -0.2607]]),
 tensor([ 0.0044, -0.0044]),
 tensor([[ 0.1115, -0.0859],
         [-0.2387,  0.1840]]),
 tensor([-0.0020,  0.0043]),
 tensor([[-0.1377,  0.2607],
         [ 0.1377, -0.2607]]),
 tensor([ 0.0044, -0.0044]),
 tensor([[-0.1780,  0.1051],
         [ 0.3713, -0.1461]]),
 tensor([-0.4169,  0.5672]),
 tensor([[ 0.2006, -0.2316],
         [-0.7491,  0.3455]]),
 tensor([ 0.1171, -0.8668]))

## 3) loss에 3을 곱해줘야 내가 원하는 값을 얻을 수 있다

In [9]:
ce_gradient =  torch.autograd.grad(torch.tensor(3) * ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(grad_param)

tensor([[ 0.3345, -0.2578],
        [-0.7161,  0.5520]])
tensor([-0.0061,  0.0130])
tensor([[-0.4132,  0.7822],
        [ 0.4132, -0.7822]])
tensor([ 0.0132, -0.0132])


# 3. 각 parameter 별로 3을 곱해준다

In [10]:
# Gradient 확인
print("Cross Entropy Loss Gradient:")
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(torch.tensor(3) * grad_param)

Cross Entropy Loss Gradient:
tensor([[ 0.3345, -0.2578],
        [-0.7161,  0.5520]])
tensor([-0.0061,  0.0130])
tensor([[-0.4132,  0.7822],
        [ 0.4132, -0.7822]])
tensor([ 0.0132, -0.0132])


## 4. weight에 2를 곱하고 gradient를 구하면 값이 달라진다

In [21]:
# 각 레이어의 가중치에 2를 곱함
for param in model.parameters():
    param.data = param.data * 2

ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(grad_param)

tensor([[ 0.1115, -0.0859],
        [-0.2387,  0.1840]])
tensor([-0.0020,  0.0043])
tensor([[-0.1377,  0.2607],
        [ 0.1377, -0.2607]])
tensor([ 0.0044, -0.0044])


In [24]:
ce_loss = torch.tensor(0.1, requires_grad=True)
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)

print(ce_gradient)

for grad_param in ce_gradient:
    print(grad_param)

RuntimeError: One of the differentiated Tensors appears to not have been used in the graph. Set allow_unused=True if this is the desired behavior.